In [6]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

In [2]:
data = pd.read_csv("input/GSAF5.csv",encoding='cp1252')
display(data)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN


In [34]:
paises_datos = (data["Country"].value_counts())
paises_datos.sort_values(ascending=False).head(20)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
BRAZIL               102
BAHAMAS               98
MEXICO                81
ITALY                 71
FIJI                  62
PHILIPPINES           59
REUNION               57
NEW CALEDONIA         51
MOZAMBIQUE            44
CUBA                  42
SPAIN                 40
INDIA                 37
EGYPT                 36
CROATIA               34
PANAMA                32
Name: Country, dtype: int64

In [63]:
lesiones = data["Injury"].value_counts()
lesiones.sort_values(ascending=False).head(60)

FATAL                               729
Survived                             97
Foot bitten                          82
No injury                            77
Leg bitten                           71
Left foot bitten                     48
No details                           43
Right foot bitten                    39
Hand bitten                          28
No injury, board bitten              27
Thigh bitten                         27
FATAL, body not recovered            24
Minor injury                         22
Foot lacerated                       21
Calf bitten                          21
Right leg bitten                     20
Arm bitten                           19
Lacerations to foot                  19
Ankle bitten                         17
Right calf bitten                    16
Lacerations to right foot            15
No injury to occupants               14
Lacerations to left foot             14
Left arm bitten                      13
Foot severed                         13


In [49]:
australia = data[(data["Country"]== "AUSTRALIA")]
australia["Species "].value_counts()

White shark                                                              41
Wobbegong shark                                                          18
Grey nurse shark                                                         11
3 m [10'] shark                                                          11
Bull shark                                                               10
                                                                         ..
Tiger shark, 4.3 m [14'], 3 tooth fragments retrieved from his wounds     1
Large tiger shark seen in the vicinity the following morning              1
"reef shark"                                                              1
4.5 m & 5 m white shark                                                   1
2.1 m [7'] shark                                                          1
Name: Species , Length: 415, dtype: int64

In [70]:
reducida = australia[["Injury", "Type", "Activity"]]
reducida.sort_values("Type", ascending=False).head(200)

,Injury,Type,Activity
3,Struck by fin on chest & leg,Unprovoked,Surfing
4642,Torso bitten,Unprovoked,Pearl diving
4540,Minor injuries to foot,Unprovoked,Free diving for lobster
4543,"FATAL, injuries to both arms",Unprovoked,Wading
4544,Shoulder nipped,Unprovoked,Dived into the water
...,...,...,...
4285,"Left arm severed above elbow, recovered 5 days...",Unprovoked,Bathing
3876,Swim fin bitten,Unprovoked,Spearfishing
4134,Bruised shoulder chest & foot,Unprovoked,Swimming or wading out to warn bathers that a...
4135,"No Injury, shark charged surfski",Unprovoked,Surf skiing
